In [ ]:
import sys
!python -m pip install sentinelhub

In [ ]:
# code for NDVI for a single image
from sentinelhub import (
    SHConfig,
    DataCollection,
    SentinelHubRequest,
    MimeType,
    bbox_to_dimensions,
    BBox,
    CRS,
)

# Step 1: Set up your configuration
config = SHConfig()
config.sh_client_id = 'sh-f4a2dffd-e52c-4d3b-9e5b-e1a2787df9b6'  # Replace with your Sentinel Hub client ID
config.sh_client_secret = 'hUZYxAyDlH9WEb3GTnLvowHkJLnBkios'  # Replace with your Sentinel Hub client secret
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"

# Step 2: Define request parameters
# Define the bounding box (location) in WGS84 format (latitude, longitude)
# Example coordinates: Adjust as needed
bbox = BBox((12.27, 45.52, 12.30, 45.55), crs=CRS.WGS84)  # Example coordinates

# Define the resolution (in meters) and calculate the image size
resolution = 10  # 10 m resolution
size = bbox_to_dimensions(bbox, resolution)

# Define the time interval (only May 1st, 2020)
time_interval = ('2020-05-01', '2020-05-03')

# Define the Evalscript for NDVI
evalscript = """
//VERSION=3

function setup() {
    return {
        input: ["B04", "B08"],
        output: { id: "default", bands: 1, sampleType: "FLOAT32" }
    };
}

function evaluatePixel(sample) {
    const ndvi = (sample.B08 - sample.B04) / (sample.B08 + sample.B04);
    return [ndvi];
}
"""

# Step 3: Create the SentinelHubRequest
request = SentinelHubRequest(
    evalscript=evalscript,
    input_data=[
        SentinelHubRequest.input_data(
            DataCollection.SENTINEL2_L1C.define_from(
                "s2l1c", service_url=config.sh_base_url
            ),
            time_interval=time_interval,
        )
    ],
    responses=[SentinelHubRequest.output_response('default', MimeType.TIFF)],
    bbox=bbox,
    size=size,
    config=config,
)

# Step 4: Execute the request and download the NDVI image
ndvi_image = request.get_data()[0]

# Step 5: Save the downloaded NDVI image to a file
output_file_path = 'ndvi_image_1.tiff'
with open(output_file_path, 'wb') as f:
    f.write(ndvi_image)

print(f"NDVI image downloaded and saved to: {output_file_path}")


In [44]:
#code for requesting all bands for a image
import datetime
import os
import getpass

import matplotlib.pyplot as plt
import numpy as np

from sentinelhub import (
    SHConfig,
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
)

# Step 1: Set up your configuration
config = SHConfig()
config.sh_client_id = 'sh-f4a2dffd-e52c-4d3b-9e5b-e1a2787df9b6'  # Replace with your Sentinel Hub client ID
config.sh_client_secret = 'hUZYxAyDlH9WEb3GTnLvowHkJLnBkios'  # Replace with your Sentinel Hub client secret
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"


betsiboka_coords_wgs84 = (46.16, -16.15, 46.51, -15.58)
resolution = 60
betsiboka_bbox = BBox(bbox=betsiboka_coords_wgs84, crs=CRS.WGS84)
betsiboka_size = bbox_to_dimensions(betsiboka_bbox, resolution=resolution)

print(f"Image shape at {resolution} m resolution: {betsiboka_size} pixels")

evalscript_all_bands = """
    //VERSION=3
    function setup() {
        return {
            input: [{
                bands: ["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B10","B11","B12"],
                units: "DN"
            }],
            output: {
                bands: 13,
                sampleType: "INT16"
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B01,
                sample.B02,
                sample.B03,
                sample.B04,
                sample.B05,
                sample.B06,
                sample.B07,
                sample.B08,
                sample.B8A,
                sample.B09,
                sample.B10,
                sample.B11,
                sample.B12];
    }
"""

request_all_bands = SentinelHubRequest(
    data_folder="test_dir",
    evalscript=evalscript_all_bands,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L1C.define_from(
                "s2l1c", service_url=config.sh_base_url
            ),
            time_interval=("2020-06-01", "2020-06-01"),
            mosaicking_order=MosaickingOrder.LEAST_CC,
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
    bbox=betsiboka_bbox,
    size=betsiboka_size,
    config=config,
)

all_bands_response = request_all_bands.get_data(save_data=True)



Image shape at 60 m resolution: (631, 1047) pixels


In [51]:
#code for requesting all bands for for multiple days (multiple timestamp)
import datetime
import os
import getpass

import matplotlib.pyplot as plt
import numpy as np

from sentinelhub import (
    SHConfig,
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
)

# Step 1: Set up your configuration
config = SHConfig()
config.sh_client_id = 'sh-f4a2dffd-e52c-4d3b-9e5b-e1a2787df9b6'  # Replace with your Sentinel Hub client ID
config.sh_client_secret = 'hUZYxAyDlH9WEb3GTnLvowHkJLnBkios'  # Replace with your Sentinel Hub client secret
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"


betsiboka_coords_wgs84 = (46.16, -16.15, 46.51, -15.58)
resolution = 60
betsiboka_bbox = BBox(bbox=betsiboka_coords_wgs84, crs=CRS.WGS84)
betsiboka_size = bbox_to_dimensions(betsiboka_bbox, resolution=resolution)

print(f"Image shape at {resolution} m resolution: {betsiboka_size} pixels")


start = datetime.datetime(2019, 1, 1)
end = datetime.datetime(2019, 12, 31)
n_chunks = 13
tdelta = (end - start) / n_chunks
edges = [(start + i * tdelta).date().isoformat() for i in range(n_chunks)]
slots = [(edges[i], edges[i + 1]) for i in range(len(edges) - 1)]

print("Monthly time windows:\n")
for slot in slots:
    print(slot)
evalscript_true_color = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B02", "B03", "B04"]
            }],
            output: {
                bands: 3
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B04, sample.B03, sample.B02];
    }
"""

request_true_color = SentinelHubRequest(
    data_folder='test_dir',
    evalscript=evalscript_true_color,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L1C.define_from(
                "s2l1c", service_url=config.sh_base_url
            ),
            time_interval=("2020-06-12", "2020-06-13"),
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
    bbox=betsiboka_bbox,
    size=betsiboka_size,
    config=config,
)
def get_true_color_request(time_interval):
    return SentinelHubRequest(
        data_folder="test_dir",
        evalscript=evalscript_true_color,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L1C.define_from(
                    "s2l1c", service_url=config.sh_base_url
                ),
                time_interval=time_interval,
                mosaicking_order=MosaickingOrder.LEAST_CC,
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
        bbox=betsiboka_bbox,
        size=betsiboka_size,
        config=config,
    )
# create a list of requests
list_of_requests = [get_true_color_request(slot).get_data(save_data=True) for slot in slots]
#list_of_requests = [request.download_list[0] for request in list_of_requests]

# download data with multiple threads
#data = SentinelHubDownloadClient(config=config).download(
 #   list_of_requests.get_data(save_data=True), max_threads=5
#)


Image shape at 60 m resolution: (631, 1047) pixels
Monthly time windows:

('2019-01-01', '2019-01-29')
('2019-01-29', '2019-02-26')
('2019-02-26', '2019-03-26')
('2019-03-26', '2019-04-23')
('2019-04-23', '2019-05-21')
('2019-05-21', '2019-06-18')
('2019-06-18', '2019-07-16')
('2019-07-16', '2019-08-13')
('2019-08-13', '2019-09-10')
('2019-09-10', '2019-10-08')
('2019-10-08', '2019-11-05')
('2019-11-05', '2019-12-03')


In [ ]:
#code for requesting all bands for for multiple days (multiple timestamp)
import datetime
import os
import getpass

import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

from sentinelhub import (
    SHConfig,
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
)

# Step 1: Set up your configuration
config = SHConfig()
config.sh_client_id = 'sh-f4a2dffd-e52c-4d3b-9e5b-e1a2787df9b6'  # Replace with your Sentinel Hub client ID
config.sh_client_secret = 'hUZYxAyDlH9WEb3GTnLvowHkJLnBkios'  # Replace with your Sentinel Hub client secret
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"


betsiboka_coords_wgs84 = (46.16, -16.15, 46.51, -15.58)
resolution = 60
betsiboka_bbox = BBox(bbox=betsiboka_coords_wgs84, crs=CRS.WGS84)
betsiboka_size = bbox_to_dimensions(betsiboka_bbox, resolution=resolution)

print(f"Image shape at {resolution} m resolution: {betsiboka_size} pixels")


start = datetime.datetime(2018, 1, 1)
end = datetime.datetime(2018, 12, 31)
n_chunks = 13
tdelta = (end - start) / n_chunks
edges = [(start + i * tdelta).date().isoformat() for i in range(n_chunks)]
slots = [(edges[i], edges[i + 1]) for i in range(len(edges) - 1)]

print("Monthly time windows:\n")
for slot in slots:
    print(slot)
evalscript_true_color = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B10","B11","B12"]
            }],
            output: {
                bands: 12
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B01,
                sample.B02,
                sample.B03,
                sample.B04,
                sample.B05,
                sample.B06,
                sample.B07,
                sample.B08,
                sample.B8A,
                sample.B09,
                sample.B10,
                sample.B11,
                sample.B12];
    }
"""

request_true_color = SentinelHubRequest(
    data_folder="C:/Users/amusu/OneDrive/Documents/Changedetection/dataset_sentinel2",
    evalscript=evalscript_true_color,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L1C.define_from(
                "s2l1c", service_url=config.sh_base_url
            ),
            time_interval=("2018-01-01", "2018-12-31"),
            mosaicking_order=MosaickingOrder.LEAST_CC,
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
    bbox=betsiboka_bbox,
    size=betsiboka_size,
    config=config,
)
def get_true_color_request(time_interval):
    return SentinelHubRequest(
        data_folder="C:/Users/amusu/OneDrive/Documents/Changedetection/dataset_sentinel2",
        evalscript=evalscript_true_color,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L1C.define_from(
                    "s2l1c", service_url=config.sh_base_url
                ),
                time_interval=time_interval,
                mosaicking_order=MosaickingOrder.LEAST_CC,
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
        bbox=betsiboka_bbox,
        size=betsiboka_size,
        config=config,
    )
# create a list of requests
list_of_requests = [get_true_color_request(slot).get_data(save_data=True) for slot in slots]
#list_of_requests = [request.download_list[0] for request in list_of_requests]

# download data with multiple threads
#data = SentinelHubDownloadClient(config=config).download(
 #   list_of_requests.get_data(save_data=True), max_threads=5
#)

In [ ]:
#code for requesting all bands for for multiple days (multiple timestamp) for ROME
import datetime
import os
import getpass

import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

from sentinelhub import (
    SHConfig,
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
)

# Step 1: Set up your configuration
config = SHConfig()
config.sh_client_id = 'sh-f4a2dffd-e52c-4d3b-9e5b-e1a2787df9b6'  # Replace with your Sentinel Hub client ID
config.sh_client_secret = 'hUZYxAyDlH9WEb3GTnLvowHkJLnBkios'  # Replace with your Sentinel Hub client secret
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"


#betsiboka_coords_wgs84 = (46.16, -16.15, 46.51, -15.58)
betsiboka_coords_wgs84 = ( 12.26,41.77,12.64,42.03)
resolution = 60
betsiboka_bbox = BBox(bbox=betsiboka_coords_wgs84, crs=CRS.WGS84)
betsiboka_size = bbox_to_dimensions(betsiboka_bbox, resolution=resolution)

print(f"Image shape at {resolution} m resolution: {betsiboka_size} pixels")


start = datetime.datetime(2018, 1, 1)
end = datetime.datetime(2018, 12, 31)
n_chunks = 13
tdelta = (end - start) / n_chunks
edges = [(start + i * tdelta).date().isoformat() for i in range(n_chunks)]
slots = [(edges[i], edges[i + 1]) for i in range(len(edges) - 1)]

print("Monthly time windows:\n")
for slot in slots:
    print(slot)
evalscript_true_color = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B02", "B03", "B04","B08"]
            }],
            output: {
                bands: 4
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B08, sample.B04, sample.B03, sample.B02];
    }
"""

request_true_color = SentinelHubRequest(
    data_folder='test_dir',
    evalscript=evalscript_true_color,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L1C.define_from(
                "s2l1c", service_url=config.sh_base_url
            ),
            time_interval=("2018-01-01", "2018-12-31"),
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
    bbox=betsiboka_bbox,
    size=betsiboka_size,
    config=config,
)
def get_true_color_request(time_interval):
    return SentinelHubRequest(
        data_folder="test_dir",
        evalscript=evalscript_true_color,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L1C.define_from(
                    "s2l1c", service_url=config.sh_base_url
                ),
                time_interval=time_interval,
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
        bbox=betsiboka_bbox,
        size=betsiboka_size,
        config=config,
    )
# create a list of requests
list_of_requests = [get_true_color_request(slot).get_data(save_data=True) for slot in slots]